# Constraint Docking

This feature is available since version 2.0.0

**NOTE**: Since version 2.1.7 it is not needed to specify `protected_ids` to avoid failures of **MolDrug**. However, on this tutorial we will let the workaround for user with older versions. May be that the specified CReM parameters completely remove any similarity between a solution and the reference structure for constraint docking; only in this specific scenarios you should protect some atoms in order to preserve some MCS.

In [1]:
import tempfile, os, requests, gzip, shutil, yaml
from multiprocessing import cpu_count
from moldrug.data import ligands, boxes, receptor_pdbqt, receptor_pdb, constraintref
from moldrug import utils
tmp_path = tempfile.TemporaryDirectory()
import numpy as np
from rdkit import Chem

In [2]:
def atom_ids_list(smiles:str) -> list[int]:
    """Return a list of atom IDs for the molecule

    Parameters
    ----------
    smiles : str
        The SMILES string of the molecule

    Returns
    -------
    list[int]
        List of atoms IDs
    """
    return list(range(Chem.MolFromSmiles(smiles).GetNumAtoms()))

In [3]:
# Setting the working directory (you could change it but it MUST be an absolute path)
wd = tmp_path.name
# os.makedirs('wd_tutorial', exist_ok=True)
# wd = os.path.abspath('wd_tutorial')


In [4]:
# Getting the data
lig = ligands.r_x0161
box = boxes.r_x0161
with open(os.path.join(wd, 'x0161.pdbqt'), 'w') as f:
    f.write(receptor_pdbqt.r_x0161)
with open(os.path.join(wd, 'x0161.pdb'), 'w') as f:
    f.write(receptor_pdb.r_x0161)
with open(os.path.join(wd, 'ref.sdf'), 'w') as f:
    f.write(constraintref.r_x0161)
# Getting the CReM data base
url = "http://www.qsar4u.com/files/cremdb/replacements02_sc2.db.gz"
r = requests.get(url, allow_redirects=True)
crem_dbgz_path = os.path.join(wd,'crem.db.gz')
crem_db_path = os.path.join(wd,'crem.db')
open(crem_dbgz_path, 'wb').write(r.content)
with gzip.open(crem_dbgz_path, 'rb') as f_in:
    with open(crem_db_path, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

print(lig)
print(atom_ids_list(lig))
print(box)
print(wd, os.listdir(wd))

COC(=O)C=1C=CC(=CC1)S(=O)(=O)N
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
{'A': {'boxcenter': [12.11, 1.84, 23.56], 'boxsize': [22.5, 22.5, 22.5]}}
/var/folders/fl/txtwcjh94vs_gkm_s21h8w7m0000gn/T/tmphs9lvmia ['x0161.pdbqt', 'crem.db.gz', 'ref.sdf', 'crem.db', 'x0161.pdb']


## Setting the configuration

**MolDrug**'s version 2.0.4 tried to fix some issues with constraint docking ([see release description](https://moldrug.readthedocs.io/en/latest/source/CHANGELOG.html)). If follow jobs are needed and `mutate_crem_kwargs` allows changes in heavy atoms; it *MUST* be specified the keyword `protected_ids` for `mutate_crem_kwargs`. This is needed because during the generation of constraint conformers `constraint_ref` is used as fix core. Therefore if a generated molecule does not have this core, an error happens. On version 2.0.0 the core was guessed based on MCS but some error happened ([see this RDKit bug](https://github.com/rdkit/rdkit/issues/5518)). You have then two possibles work around:

1. Use `mutate_crem_kwargs` that only allow grow operations (`min_size = max_size = 0`). In this way the core will be preserved.
2. Specify `protected_ids` inside `mutate_crem_kwargs`. Those IDs are the atom indexes of `seed_mol` that correspond to the atoms of `constraint_ref`.

On this tutorial we will use the second strategy. In this case `constraint_ref` is the same as `seed_mol` but with some specific conformation state. Therefore we could use the the result of `atom_ids_list` function (specified above) to get the protected_ids list of atoms. 

In [5]:
config ={
    "01_grow": {
        "type": "GA",
        "njobs": 6,
        "seed_mol": lig,
        "AddHs": True,
        "costfunc": "Cost",
        "costfunc_kwargs": {
            "vina_executable": "/Users/klimt/GIT/docking/vina_1.2.5_mac_x86_64", # You should change to the proper vina executable
            "receptor_pdbqt_path": os.path.join(wd, 'x0161.pdbqt'),
            "boxcenter": box['A']['boxcenter'],
            "boxsize": box['A']['boxsize'],
            "exhaustiveness": 4,
            "ncores": int(cpu_count() / 6),
            "num_modes": 1,
            "constraint": True,
            "constraint_type": "score_only", # local_only
            "constraint_ref": os.path.join(wd, 'ref.sdf'),
            "constraint_receptor_pdb_path": os.path.join(wd, 'x0161.pdb'),
            "constraint_num_conf": 100,
            "constraint_minimum_conf_rms": 0.01
        },
        "crem_db_path": crem_db_path,
        "maxiter": 2,
        "popsize": 10,
        "beta": 0.001,
        "pc": 0.5,
        "get_similar": False,
        "mutate_crem_kwargs": {
            "radius": 3,
            "min_size": 0,
            "max_size": 0,
            "min_inc": -5,
            "max_inc": 6,
            "protected_ids": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
            "ncores": 12
        },
        "save_pop_every_gen": 10,
        "deffnm": "01_grow"
    },
    "02_allow_grow": {
        "mutate_crem_kwargs": {
            "radius": 3,
            "min_size": 0,
            "max_size": 2,
            "min_inc": -5,
            "max_inc": 3,
            "protected_ids": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
            "ncores": 12
        },
        "maxiter": 2,
        "deffnm": "02_allow_grow"
    },
    "03_pure_mutate": {
        "mutate_crem_kwargs": {
            "radius": 3,
            "min_size": 1,
            "max_size": 8,
            "min_inc": -5,
            "max_inc": 3,
            "protected_ids": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
            "ncores": 12
        },
        "maxiter": 2,
        "deffnm": "03_pure_mutate"
    },
    "04_local": {
        "mutate_crem_kwargs": {
            "radius": 3,
            "min_size": 0,
            "max_size": 1,
            "min_inc": -1,
            "max_inc": 1,
            "protected_ids": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
            "ncores": 12
        },
        "maxiter": 2,
        "deffnm": "04_local"
    }
}
# Save the config as a yaml file
with open(os.path.join(wd, 'config.yml'), 'w') as f:
    yaml.dump(config, f)
os.listdir(wd)

['x0161.pdbqt', 'crem.db.gz', 'ref.sdf', 'config.yml', 'crem.db', 'x0161.pdb']

## Move to the directory and run moldrug

In [6]:
cwd = os.getcwd()
os.chdir(wd)
! moldrug config.yml
os.chdir(cwd)
os.listdir(wd)
os.chdir(cwd)

Started at Mon Sep  4 18:24:41 2023
You are using moldrug: 3.4.0.

CommandLineHelper(yaml_file='config.yml', fitness=None, outdir=None, continuation=False)




Creating the first population with 10 members:
  0%|                                                    | 0/10 [00:00<?, ?it/s]/opt/homebrew/Caskroom/miniconda/base/envs/moldrug/lib/python3.9/site-packages/meeko/preparation.py:230: DeprecationWarning: MoleculePreparation.write_pdbqt_string() is deprecated in Meeko v0.5. Pass the MoleculeSetup instance to PDBQTWriterLegacy.write_string(). MoleculePreparation.prepare() returns a list of MoleculeSetup instances.
  warnings.warn(msg, DeprecationWarning)
/opt/homebrew/Caskroom/miniconda/base/envs/moldrug/lib/python3.9/site-packages/meeko/preparation.py:93: DeprecationWarning: MoleculePreparation.setup is deprecated in Meeko v0.5. MoleculePreparation.prepare() returns a list of MoleculeSetup instances.
  warnings.warn(msg, DeprecationWarning)
 10%|████▍                                

In [7]:
os.listdir(wd)

['02_allow_grow_pop.sdf',
 '04_local_pop.sdf',
 '01_grow_result.pbz2',
 'x0161.pdbqt',
 '03_pure_mutate_pop.sdf',
 '04_local_result.pbz2',
 '01_grow_pop.sdf',
 '03_pure_mutate_result.pbz2',
 'crem.db.gz',
 '03_pure_mutate_pop.pbz2',
 'ref.sdf',
 'config.yml',
 'error.tar.gz',
 '02_allow_grow_result.pbz2',
 'crem.db',
 '01_grow_pop.pbz2',
 '02_allow_grow_pop.pbz2',
 '04_local_pop.pbz2',
 'x0161.pdb']

## Visualization

If needed install the packages through pip:

`! pip install nglview, parmed`


In [8]:
# import nglview as nv
# import parmed as pmd
# from ipywidgets import IntSlider, VBox

In [9]:
# parms = pmd.rdkit.from_sdf(os.path.join(wd, '04_local_pop.sdf')) # require parmed, rdkit
# parms

[<Structure 23 atoms; 1 residues; 23 bonds; NOT parameterized>,
 <Structure 36 atoms; 1 residues; 36 bonds; NOT parameterized>,
 <Structure 26 atoms; 1 residues; 26 bonds; NOT parameterized>,
 <Structure 37 atoms; 1 residues; 37 bonds; NOT parameterized>,
 <Structure 29 atoms; 1 residues; 29 bonds; NOT parameterized>,
 <Structure 34 atoms; 1 residues; 34 bonds; NOT parameterized>,
 <Structure 34 atoms; 1 residues; 34 bonds; NOT parameterized>,
 <Structure 29 atoms; 1 residues; 29 bonds; NOT parameterized>,
 <Structure 34 atoms; 1 residues; 34 bonds; NOT parameterized>,
 <Structure 32 atoms; 1 residues; 32 bonds; NOT parameterized>]

In [10]:
# view = nv.NGLWidget()

# slider = IntSlider(max=len(parms)-1)

# def show_one_ligand(change):
#     val = change['new']
#     view.show_only([val])
    
# slider.observe(show_one_ligand, 'value')

# VBox([view, slider])

In [11]:
# for p in parms:
#     view.add_structure(nv.ParmEdTrajectory(p))
# view.show_only([0])

In [12]:
result = utils.decompress_pickle(os.path.join(wd, '04_local_result.pbz2'))
valid_individuals_vina_score = [individual.vina_score for individual in result.pop if individual.vina_score != np.inf]
print(f"avg = {np.average(valid_individuals_vina_score)}")
print(f"min = {min(valid_individuals_vina_score)}")
print(f"max = {max(valid_individuals_vina_score)}")

avg = -3.5055
min = -4.942
max = -0.549


## See how it looks with local_only

In [13]:
# Save the new config as a yaml file
new_config = config.copy()
new_config['01_grow']['costfunc_kwargs']['constraint_type'] = 'local_only'

with open(os.path.join(wd, 'new_config.yml'), 'w') as f:
    yaml.dump(config, f)
os.listdir(wd)

['02_allow_grow_pop.sdf',
 '04_local_pop.sdf',
 'new_config.yml',
 '01_grow_result.pbz2',
 'x0161.pdbqt',
 '03_pure_mutate_pop.sdf',
 '04_local_result.pbz2',
 '01_grow_pop.sdf',
 '03_pure_mutate_result.pbz2',
 'crem.db.gz',
 '03_pure_mutate_pop.pbz2',
 'ref.sdf',
 'config.yml',
 'error.tar.gz',
 '02_allow_grow_result.pbz2',
 'crem.db',
 '01_grow_pop.pbz2',
 '02_allow_grow_pop.pbz2',
 '04_local_pop.pbz2',
 'x0161.pdb']

## Move back to the directory and run moldrug

In [14]:
cwd = os.getcwd()
os.chdir(wd)
! moldrug new_config.yml
os.chdir(cwd)
os.listdir(wd)
os.chdir(cwd)

Started at Mon Sep  4 18:32:14 2023
You are using moldrug: 3.4.0.

CommandLineHelper(yaml_file='new_config.yml', fitness=None, outdir=None, continuation=False)




Creating the first population with 10 members:
100%|███████████████████████████████████████████| 10/10 [00:52<00:00,  5.20s/it]
Initial Population: Best Individual: Individual(idx = 8, smiles = COC(=O)c1ccc(S(=O)(=O)NN=C(N)N)cc1, cost = 1.0)
Accepted rate: 10 / 10

File 01_grow_pop.sdf was createad!
Evaluating generation 1 / 2:
100%|█████████████████████████████████████████████| 5/5 [01:19<00:00, 16.00s/it]
Generation 1: Best Individual: Individual(idx = 12, smiles = NC(N)=NNS(=O)(=O)c1ccc(C(=O)OCC(F)(F)F)cc1, cost = 0.8812342926598279).
Accepted rate: 1 / 5

Evaluating generation 2 / 2:
100%|█████████████████████████████████████████████| 5/5 [01:39<00:00, 19.89s/it]
File 01_grow_pop.sdf was createad!
Generation 2: Best Individual: Individual(idx = 12, smiles = NC(N)=NNS(=O)(=O)c1ccc(C(=O)OCC(F)(F)F)cc1, cost = 0.88123429265

In [15]:
# parms = pmd.rdkit.from_sdf(os.path.join(wd, '04_local_pop.sdf')) # require parmed, rdkit
# parms

In [16]:
# view = nv.NGLWidget()

# slider = IntSlider(max=len(parms)-1)

# def show_one_ligand(change):
#     val = change['new']
#     view.show_only([val])
    
# slider.observe(show_one_ligand, 'value')

# VBox([view, slider])

In [17]:
# for p in parms:
#     view.add_structure(nv.ParmEdTrajectory(p))
# view.show_only([0])

In [18]:
result = utils.decompress_pickle(os.path.join(wd, '04_local_result.pbz2'))
valid_individuals_vina_score = [individual.vina_score for individual in result.pop if individual.vina_score != np.inf]
print(f"avg = {np.average(valid_individuals_vina_score)}")
print(f"min = {min(valid_individuals_vina_score)}")
print(f"max = {max(valid_individuals_vina_score)}")


avg = -5.402699999999999
min = -6.342
max = -4.724


As you can see now the fix core changes a little. On both examples (when you run the tutorial by yourself may be different) **MolDrug** can not optimize the cost function. Even so, the local_only strategy performs better respect to maximum and average cost. One of the main cause of this behavior is the desirability definition which is for vina_score:

```
'vina_score': {
    'w': 1,
    'SmallerTheBest': {
        'Target': -12,
        'UpperLimit': -6,
        'r': 1
    }
}
```

This means that if the vina score is not lower than -6 the cost function will be always 1. In this scenario all were are not optimizing over the generations the "best" individuals. In other words, it is important to see what is the behavior of our system (one small simulation) and then tune your parameters accordantly. On possibility for this system is change UpperLimit to, for example -4. There are a lot of other different options that you could play around (e.g. increase the generations, etc...). It is important to remark that this example is just for the tutorial, in a real project you may need to increase generations, tune the crem keywords, etc. I let it to you as a challenge. Happy simulations!

Just as a bonus I let you here how to change the desirability definition.

## Change the desirability

In [19]:
# Save the new config as a yaml file
tune_config = new_config.copy()
tune_config['01_grow']['costfunc_kwargs']['desirability'] = {
        'qed': {
            'w': 1,
            'LargerTheBest': {
                'LowerLimit': 0.1,
                'Target': 0.75,
                'r': 1
            }
        },
        'sa_score': {
            'w': 1,
            'SmallerTheBest': {
                'Target': 3,
                'UpperLimit': 7,
                'r': 1
            }
        },
        'vina_score': {
            'w': 1,
            'SmallerTheBest': {
                'Target': -8,
                'UpperLimit': -3,
                'r': 1
            }
        }
    }

with open(os.path.join(wd, 'tune_config.yml'), 'w') as f:
    yaml.dump(config, f)
os.listdir(wd)


['02_allow_grow_pop.sdf',
 '04_local_pop.sdf',
 'new_config.yml',
 '01_grow_result.pbz2',
 'x0161.pdbqt',
 '03_pure_mutate_pop.sdf',
 '04_local_result.pbz2',
 '01_grow_pop.sdf',
 '03_pure_mutate_result.pbz2',
 'crem.db.gz',
 '03_pure_mutate_pop.pbz2',
 'tune_config.yml',
 'ref.sdf',
 'config.yml',
 'error.tar.gz',
 '02_allow_grow_result.pbz2',
 'crem.db',
 '01_grow_pop.pbz2',
 '02_allow_grow_pop.pbz2',
 '04_local_pop.pbz2',
 'x0161.pdb']

In [20]:

cwd = os.getcwd()
os.chdir(wd)
! moldrug tune_config.yml
os.chdir(cwd)
os.listdir(wd)
os.chdir(cwd)

Started at Mon Sep  4 18:45:47 2023
You are using moldrug: 3.4.0.

CommandLineHelper(yaml_file='tune_config.yml', fitness=None, outdir=None, continuation=False)




Creating the first population with 10 members:
100%|███████████████████████████████████████████| 10/10 [01:12<00:00,  7.27s/it]
Initial Population: Best Individual: Individual(idx = 1, smiles = N#CCOC(=O)c1ccc(S(N)(=O)=O)cc1, cost = 0.5520254583868194)
Accepted rate: 10 / 10

File 01_grow_pop.sdf was createad!
Evaluating generation 1 / 2:
100%|█████████████████████████████████████████████| 5/5 [01:32<00:00, 18.58s/it]
Generation 1: Best Individual: Individual(idx = 1, smiles = N#CCOC(=O)c1ccc(S(N)(=O)=O)cc1, cost = 0.5520254583868194).
Accepted rate: 4 / 5

Evaluating generation 2 / 2:
100%|█████████████████████████████████████████████| 5/5 [01:23<00:00, 16.72s/it]
File 01_grow_pop.sdf was createad!
Generation 2: Best Individual: Individual(idx = 1, smiles = N#CCOC(=O)c1ccc(S(N)(=O)=O)cc1, cost = 0.5520254583868194).
Accept

In [21]:
result = utils.decompress_pickle(os.path.join(wd, '04_local_result.pbz2'))
valid_individuals_vina_score = [individual.vina_score for individual in result.pop if individual.vina_score != np.inf]
print(f"avg = {np.average(valid_individuals_vina_score)}")
print(f"min = {min(valid_individuals_vina_score)}")
print(f"max = {max(valid_individuals_vina_score)}")


avg = -5.1836
min = -5.727
max = -4.8


As you see the maximum is much lower than the rest of the examples. So, we get some improvements changing the desirability.